In [6]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 구글맵의 '뱅뱅막국수' 페이지로 이동
url = "https://www.google.com/maps/search/뱅뱅막국수"
driver.get(url)

# 특정 요소가 로드될 때까지 기다림
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "m6QErb")))

# 모든 리뷰를 볼 수 있도록 스크롤 다운하면서 '더보기' 버튼을 클릭
while True:
    try:
        # '더보기' 버튼 클릭 (알려주신 XPath 경로 사용)
        more_reviews_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ChZDSUhNMG9nS0VJQ0FnSUNYaGUyYkZBEAE"]/span[2]/button')))
        more_reviews_button.click()
        time.sleep(2)
    except (NoSuchElementException, ElementClickInterceptedException, TimeoutException):
        # 더 이상 '더보기' 버튼이 없으면 반복 종료
        break

# 모든 리뷰 데이터를 수집
reviews = []
review_elements = driver.find_elements(By.CLASS_NAME, "wiI7pd")

for review_element in review_elements:
    try:
        # 리뷰 텍스트 추출
        review_text = review_element.text
        reviews.append(review_text)
    except:
        pass

# CSV 파일로 저장
reviews_df = pd.DataFrame(reviews, columns=["Review"])
reviews_df.to_csv("bangbang_makguksu_reviews.csv", index=False, encoding='utf-8-sig')

# 데이터프레임 출력
print(reviews_df)

# 드라이버 종료
driver.quit()


                                              Review
0  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...


In [8]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

try:
    # 구글 맵 페이지 열기
    driver.get("https://www.google.com/maps")

    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)

    # 페이지 로드 대기
    time.sleep(5)

    # "리뷰 더보기" 버튼 클릭
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div/div/button[1]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기

    # 스크롤 가능한 리뷰 섹션 찾기
    scrollable_div = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
    )

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)

        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    for review in reviews:
        try:
            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text
            })
        except Exception as e:
            print(f"리뷰 수집 중 오류: {e}")
            continue

    # 데이터프레임 변환 및 출력
    reviews_df = pd.DataFrame(reviews_list)
    print(reviews_df)
    reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

except (TimeoutException, WebDriverException) as e:
    print(f"오류 발생: {e}")

finally:
    driver.quit()


NameError: name 'WebDriverException' is not defined

In [9]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, WebDriverException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

try:
    # 구글 맵 페이지 열기
    driver.get("https://www.google.com/maps")

    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)

    # 페이지 로드 대기
    time.sleep(5)

    # "리뷰 더보기" 버튼 클릭
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div/div/button[1]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기

    # 스크롤 가능한 리뷰 섹션 찾기
    scrollable_div = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
    )

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)

        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:  # 더 이상 새로운 내용이 없을 때 종료
            break
        last_height = new_height

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    for review in reviews:
        try:
            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text
            })
        except Exception as e:
            print(f"리뷰 수집 중 오류: {e}")
            continue

    # 데이터프레임 변환 및 출력
    reviews_df = pd.DataFrame(reviews_list)
    print(reviews_df)
    reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

except (TimeoutException, WebDriverException) as e:
    print(f"오류 발생: {e}")

finally:
    driver.quit()


오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x011738B3+24035]
	(No symbol) [0x010FBC44]
	(No symbol) [0x00FDC2D3]
	(No symbol) [0x0101DC86]
	(No symbol) [0x0101DECB]
	(No symbol) [0x0105B9D2]
	(No symbol) [0x0103FED4]
	(No symbol) [0x0105953F]
	(No symbol) [0x0103FC26]
	(No symbol) [0x0101218C]
	(No symbol) [0x0101310D]
	GetHandleVerifier [0x014196D3+2800643]
	GetHandleVerifier [0x0147428E+3172286]
	GetHandleVerifier [0x0146CEA2+3142610]
	GetHandleVerifier [0x01216C60+692624]
	(No symbol) [0x01104C5D]
	(No symbol) [0x01101968]
	(No symbol) [0x01101B00]
	(No symbol) [0x010F3FB0]
	BaseThreadInitThunk [0x775FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77A1809E+286]
	RtlGetAppContainerNamedObjectPath [0x77A1806E+238]



In [10]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, WebDriverException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    for _ in range(5):  # 시도 횟수 조정 가능
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break
        except:
            time.sleep(2)

    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:
            break
        last_height = new_height

scroll_down()

# 리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "ID": reviewer_id,
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  2달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 전  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...
3  Sunny Lee  3달 전  강남 뱅뱅사거리에서 막국수로 유명한 맛집입니다. 들기름막국수 강추해요. 물막국수, ...
4     schnee  4달 전  여긴 진짜 참기름막국수!!!!!!!! 정말 맛있습니다. 막국수 별로 안좋아해서,, ...
5        박경호  4달 전  아주 맛있는 국수명인이 운영하는 가게\n막국수 맛있고 만두도 맛있어요.\n기본 반찬...
6   MIN JUNG  6달 전  김치만두가 너무너무 먹고싶어서, 집근처 맛집을 알아보던중 뱅뱅막국수에 방문했습니다....
7   Jisu Yoo  8달 전  사장님과 직원분들이 너무 친절하시고, 참기름막국수, 샤브샤브, 메밀전, 육전 주문했...
8        김우현  5달 전  음식 명장 & 명인 만든 음식이라고 기대를 많이 해서 그런지 물국수는 평범했음\n참...
9        박상욱  7달 전  봉평면메밀촌 시절부터 비빔막국수에 대한 극찬을 한 적이 있는데 이 집이 이렇게나 유...


In [11]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, WebDriverException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    for _ in range(5):  # 시도 횟수 조정 가능
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break
        except:
            time.sleep(2)

    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(10):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:
            break
        last_height = new_height

scroll_down()

# 리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        # '자세히' 버튼 클릭
        try:
            more_button = review.find_element(By.XPATH, './/span[2]/button')
            more_button.click()
            time.sleep(1)  # '자세히' 클릭 후 리뷰 로드 대기
        except NoSuchElementException:
            pass  # '자세히' 버튼이 없으면 무시하고 진행
        
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "ID": reviewer_id,
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)

# 엑셀 파일로 저장
reviews_df.to_excel('bangbang_makguksu_reviews.xlsx', index=False)

# 드라이버 종료
driver.quit()


스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  2달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 전  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...
3  Sunny Lee  3달 전  강남 뱅뱅사거리에서 막국수로 유명한 맛집입니다. 들기름막국수 강추해요. 물막국수, ...
4     schnee  4달 전  여긴 진짜 참기름막국수!!!!!!!! 정말 맛있습니다. 막국수 별로 안좋아해서,, ...
5        박경호  4달 전  아주 맛있는 국수명인이 운영하는 가게\n막국수 맛있고 만두도 맛있어요.\n기본 반찬...
6   MIN JUNG  6달 전  김치만두가 너무너무 먹고싶어서, 집근처 맛집을 알아보던중 뱅뱅막국수에 방문했습니다....
7   Jisu Yoo  8달 전  사장님과 직원분들이 너무 친절하시고, 참기름막국수, 샤브샤브, 메밀전, 육전 주문했...
8        김우현  5달 전  음식 명장 & 명인 만든 음식이라고 기대를 많이 해서 그런지 물국수는 평범했음\n참...
9        박상욱  7달 전  봉평면메밀촌 시절부터 비빔막국수에 대한 극찬을 한 적이 있는데 이 집이 이렇게나 유...
